In [58]:
%load_ext autoreload
%autoreload 2

import teetool as tt
import numpy as np
import pandas as pd

In [24]:
# generate trajectory data
cluster_data_1 = tt.helpers.get_trajectories(ntype=0,
                                             ndim=2,
                                             ntraj=500,
                                             npoints=100,
                                             noise_std=0.0)

cluster_data_2 = tt.helpers.get_trajectories(ntype=1,
                                             ndim=2,
                                             ntraj=500,
                                             npoints=100,
                                             noise_std=0.0)

In [59]:
file_trajectories = "aircraft_trajectories"
# read data trajectories
df = pd.read_csv("data/{0}.csv".format(file_trajectories))

# rename columns
df.rename(columns={'EASTINGS_M': 'x [m]',
                   'NORTHINGS_M': 'y [m]',
                   'ALTITUDE_M': 'z [m]'}, inplace=True)

cluster_data_list = []

count_total = 0

for cluster_id in df.CLUSTER_ID.unique():
    # pass all clusters
    
    dfc = df[df["CLUSTER_ID"] == cluster_id]
    
    cluster_data = []

    for trajectory_id in dfc.TRAJECTORY_ID.unique():
        # pass all trajectories IN this cluster

        # extract values
        dft = dfc[dfc["TRAJECTORY_ID"] == trajectory_id]
        
        xn = dft["x [m]"].values/100.0 # metre to hectrometre (1 m = 0.001 hm)
        yn = dft["y [m]"].values/100.0 # metre to hectrometre
        
        Yn = np.array([xn, yn]).T
        
        (npoints, ndim) = Yn.shape
        
        # assume linearly distributed points (no indication otherwise)
        xn = np.linspace(0, 1, npoints)

        # add tuple to list
        cluster_data.append((xn, Yn))
    
    print("{0}   {1}".format(cluster_id, len(cluster_data)))
    
    count_total += len(cluster_data)
    
    cluster_data_list.append(cluster_data)

print("---------- + \n     {0}".format(count_total))

In [81]:
# create a world
world = tt.World(name="toy", ndim=2, resolution=[100, 100])

#for i, cluster_data in enumerate(cluster_data_list[:5]):
world.addCluster(cluster_data_list[3], "0")

# add data
#world.addCluster(cluster_data_1, "one")
# world.addCluster(cluster_data_2, "two")

In [82]:
visual = tt.visual_2d.Visual_2d(world, dpi=300)

visual.plotTrajectories(ntraj=50)
visual.show()

In [105]:
# build the model
settings = {"model_type":"ML",
            "basis_type":"bernstein",
            "nbasis": 10,
            "ngaus":100}

world.buildModel(settings)

In [106]:
# x = np.array([0.5, 0.2])
# y = np.array([[0.5], [0.2], [0.2], [0.3]])

x = np.array([1.0, 0.9, 0.8])
y = np.array([[1.0], [0.9], [0.8], [0.9], [0.85], [0.8]])

world.addObservation(icluster=0, observation = (x, y) )

In [107]:
# visual
visual = tt.visual_2d.Visual_2d(world, dpi=300)

visual.plotTube(sdwidth=2)
visual.plotSamples(ntraj=10)
visual.show()